## Imports

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
def create_csv(data_frame, file_name, rules = {}):
    for  rule,attributes in rules.items():
        for attribute in attributes:
            data_frame[attribute] = data_frame[attribute].map(rule)
    data_frame.to_csv("Csv_to_database/"+file_name, line_terminator='\r')

## Creating principal Dataframes

In [3]:
def concat_file(files):
    lfs = []
    for path in files:
        dataframe = pd.read_csv("Data/"+path, skipinitialspace=True)
        if "listing" in path :
            city_name = path.split('_')[0].capitalize()
            dataframe['city'] = city_name
        lfs.append(dataframe)
    return pd.concat(lfs)

In [4]:
listingsFiles = ["barcelona_listings.csv","berlin_listings_filtered.csv","madrid_listings_filtered.csv"]
listings = concat_file(listingsFiles).set_index('id')
listings.fillna('', inplace=True)


In [5]:
calendarFiles = ["barcelona_calendar.csv","berlin_calendar.csv","madrid_calendar.csv"]
calendars = concat_file(calendarFiles).set_index('listing_id')

In [6]:
reviewsFiles = ["barcelona_reviews.csv","berlin_reviews.csv","madrid_reviews.csv"]
reviews = concat_file(reviewsFiles).set_index('id')

## Normalize columns
Please run once this part otherwise the result will be wrong

In [7]:
def get_id(attribute):
    
    words = attribute.split('_')
    letters = [word[0] for word in words]
    return ("".join(letters).lower())+'id'

In [8]:
def find_normal_values(dataframe, attribute=''):
    if attribute != '':
        df = dataframe[attribute]
    else:
        df = dataframe
    df = df.drop_duplicates()
    df = df.dropna()
    df.index = range(len(df))
    values = df.to_dict()
    values = {v: k for k, v in values.items()}
    df = df.to_frame()
    if attribute != '':
        df = df.rename_axis(get_id(attribute))
    return values, df

In [9]:
def normalize_value(val, normalized_values):
    if (val == ''):
        return val
    return normalized_values[val]

In [10]:
def create_normalized_value(dataframe, attribute):
    normalized_values, normal_entity = find_normal_values(dataframe, attribute)
    create_csv(normal_entity, attribute+".csv")
    return dataframe[attribute].apply(normalize_value, normalized_values = normalized_values)    

In [11]:
to_normalize_attributes = ['host_response_time','room_type', 'property_type',
                           'bed_type', 'cancellation_policy', 'city']
for att in to_normalize_attributes:
    listings[att] = create_normalized_value(listings, att)

In [12]:
country_attributes = ['city', 'country', 'country_code']

df = listings[country_attributes]
df = df.drop_duplicates()
df = df.dropna()
df.index = range(len(df))
values = df[df.columns[0]].to_dict()
values = {v: k for k, v in values.items()}

listings[country_attributes[0]] =listings[country_attributes[0]].apply(normalize_value, normalized_values = values) 
listings = listings.drop(columns=country_attributes[1:])

print(df.columns.values)
df2 = df[country_attributes[1:]]
df2 = df2.drop_duplicates()
df2 = df2.dropna()
df2.index = range(len(df2))
values = df2[df2.columns[0]].to_dict()
values = {v: k for k, v in values.items()}
create_csv(df2, country_attributes[1]+".csv")
df[country_attributes[1]] =df[country_attributes[1]].apply(normalize_value, normalized_values = values) 
df = df.drop(columns=country_attributes[2:])
create_csv(df, country_attributes[0]+".csv")


['city' 'country' 'country_code']


In [13]:
#neighs = pd.concat([listings['neighbourhood','city'],listings['host_neighbourhood','city'].rename(columns={
#    'host_neighbourhood':'neighbourhood'})])
neighbourhoods = ['neighbourhood', 'city']

df = listings[neighbourhoods]
df = df.drop_duplicates()
df = df.dropna()
df.index = range(len(df))
values = df[df.columns[0]].to_dict()
values = {v: k for k, v in values.items()}
listings[neighbourhoods[0]] =listings[neighbourhoods[0]].apply(normalize_value, normalized_values = values) 
listings['host_neighbourhood'] = listings['host_neighbourhood'].apply(normalize_value, normalized_values = values)
listings = listings.drop(columns=neighbourhoods[1:])

#normalized_values, normal_entity = find_normal_values(neighbourhoods)
#create_csv(df, "neighbourhood.csv")

#listings['neighbourhood'] = listings['neighbourhood'].apply(normalize_value, normalized_values = normalized_values)
#listings['host_neighbourhood'] = listings['host_neighbourhood'].apply(normalize_value, normalized_values = normalized_values)

In [14]:
create_csv(df, "neighbourhood.csv")

In [15]:
"El Camp de l'Arpa del Clot" in listings['neighbourhood']

False

In [16]:
print(df)
print(df2)

                       neighbourhood  city
0         El Camp de l'Arpa del Clot     0
1                 La Sagrada Família     0
2                        El Poblenou     0
3                     Vila de Gràcia     0
4                   El Baix Guinardó     0
5     La Nova Esquerra de l'Eixample     0
6    Camp d'en Grassot i Gràcia Nova     0
7                Dreta de l'Eixample     0
8                           El Gòtic     0
9    L'Antiga Esquerra de l'Eixample     0
10                    La Barceloneta     0
11          Sant Pere/Santa Caterina     0
12                      El Poble-sec     0
13                     el Fort Pienc     0
14                           El Born     0
15                          El Raval     0
16       Diagonal Mar - La Mar Bella     0
17         Vallcarca i els Penitents     0
18                    Sants-Montjuïc     0
19                       Sant Antoni     0
20                           El Clot     0
21                         Pedralbes     0
22         

In [17]:
def create_list(df, attribute):
    lst = df[attribute].apply(lambda x: x[1:-1].replace('"',"").replace("'","").split(','))
    lst = lst.apply(lambda x : [y.strip(' ') for y in x])
    dct = lst.to_dict()
    tuples = []
    for key, values in dct.items():
        for value in values:
            tuples.append((key,value))

    dft = pd.DataFrame(tuples, columns=['listing_id',attribute])
    dft = dft.set_index('listing_id')
    dft = create_normalized_value(dft, attribute)
    
    dft = dft.to_frame()

    dft.to_csv("Csv_to_database/has_"+attribute+".csv")

In [18]:
create_list(listings,'amenities')
listings = listings.drop(columns=['amenities'])
create_list(listings,'host_verifications')
listings = listings.drop(columns=['host_verifications'])

In [19]:
def concat_str(val, size = 3000):
    if (val == ''):
        return val
    return str(val)[:size]

In [20]:
def convert_currency(val):
    if (val == ''):
        return val
    new_val =  str(val).replace(',','').replace('$', '')
    return float(new_val)

In [21]:
def convert_percentage(val):
    if (val == ''):
        return val
    new_val =  str(val).replace('%','')
    return int(new_val)

In [22]:
def apply_conversion(dataframe, attributes, function):
    for att in attributes:
        dataframe[att] = dataframe[att].apply(function)
    return dataframe

In [23]:
listings['host_since'] = listings['host_since'].astype('datetime64[ns]')

In [24]:
listings['host_response_rate'] = listings['host_response_rate'].apply(convert_percentage)

In [25]:
prices = ['price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee','extra_people']
for att in prices:
    listings[att] = listings[att].apply(convert_currency)

In [26]:
descriptions = ['summary', 'space', 'neighborhood_overview', 'description', 'notes', 'transit', 'access','interaction','house_rules']
for att in descriptions:
    listings[att] = listings[att].apply(concat_str, size = 500)

In [27]:
listings['host_about'] = listings['host_about'].apply(concat_str)

In [28]:
#users = []
#users.append(reviews[['user_id', 'user_name']])
#users.append(listings[['user_id', 'user_name']])
#hosts = listings[['host_id', 'host_name']]
#users = pd.concat(users).drop_duplicates()
#hosts = hosts.set_index('host_id')

In [29]:
#create_csv(hosts, "Host.csv")
#listings = listings.drop(columns=['user_name'])

In [30]:
host_columns = ['host_id', 'host_name', 'host_url', 'host_since', 'host_about', 'host_response_time',
                'host_response_rate','host_thumbnail_url','host_picture_url',
                'host_neighbourhood']
host = listings[host_columns]
host = host.set_index('host_id')
host = host[~host.index.duplicated(keep='first')]
create_csv(host, "host.csv", { concat_str: ['host_about']})
listings = listings.drop(columns=host_columns[1:])

In [31]:
create_csv(listings,"listing.csv")

## Calendar

In [32]:
att_columns = ['date', 'available', 'price']
# Selects columns from listings which appears in att_columns 
calendar = calendars[att_columns]
calendar['price'] = calendar['price'].apply(convert_currency)
#calendar.dropna(inplace=True)
calendar['date'] = calendar['date'].astype('datetime64[ns]')
calendar['price'] = calendar['price'].apply(convert_currency)
create_csv(calendar, "calendar.csv")

In [33]:
print(len(calendar))

15364310


## Reviews

In [34]:
att_columns = ['listing_id', 'reviewer_id', 'reviewer_name', 'date', 'comments']
reviews['date'] = reviews['date'].astype('datetime64[ns]')
reviews['comments'] = reviews['comments'].apply(concat_str, size = 1000)
review = reviews[att_columns]
create_csv(review,"Review.csv")

## Test


In [35]:
A = listings.index.values
B = calendar[~calendar.index.duplicated(keep='first')]
B = B.index.values
print(len(np.intersect1d(A,B)) , len(B))

42094 42094


In [36]:
A = listings.index.values
B = reviews.set_index('listing_id')
B = B[~B.index.duplicated(keep='first')]
B = B.index.values
print(len(np.intersect1d(A,B)) , len(B))

35012 35012


In [37]:
def Repeat(x): 
    _size = len(x) 
    repeated = [] 
    for i in range(_size): 
        k = i + 1
        for j in range(k, _size): 
            if x[i] == x[j] and x[i] not in repeated: 
                repeated.append(x[i]) 
    return repeated 

In [38]:
#print(Repeat(users.index.values))

In [39]:
maxi = -1
for elem in listings['description']:
    if (elem != ''):
        if (len(elem) > maxi):
            maxi = len(elem)
print(maxi)

500
